# Daily Briefing Multi Agent Example


The system consists of four agents:

1. **Retrieve User Meetings Agent** – Fetches all meetings for the day.

2. **ClickUp Tasks Agent** – Retrieves the user’s tasks from ClickUp.

3. **Industrial News Agent** – Generates work-related news posts with relevant links based on the user's tasks.

4. **Manager Agent** – Orchestrates agent interactions and finalizes the output.

at the end create a report with your schedule, highlighting key meetings, tasks, and deadlines, and pulling in relevant updates or industry news that helps the user to be more productive.

In [9]:
%pip install --upgrade xpander-sdk --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [10]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider
from os import environ
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

True

In [11]:
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]
BASE_URL = environ["BASE_URL"]

In [12]:
# load the clients
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, base_url=BASE_URL)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

In [13]:

today = datetime.now().strftime("%Y-%m-%d")
xpander_agent.add_task(f"Please create a daily briefing report for today - {today}")
xpander_agent.memory.init_messages(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)

while not xpander_agent.is_finished():
    
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.messages,
                tools=xpander_agent.get_tools(),
                tool_choice=xpander_agent.tool_choice,
                temperature=0.0
        )
    # add messages directly from the LLM response
    xpander_agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    xpander_agent.run_tools(tool_calls=tool_calls)
    

In [14]:
execution_result = xpander_agent.retrieve_execution_result()

print("status:", execution_result.status)
print("result:\n\n", execution_result.result)

status: ExecutionStatus.COMPLETED
result:

 # Daily Briefing - 2025-02-04
## Today's Schedule
**Meetings:**
- 07:30 - 08:30: demo: xpander.ai platform (Attendees: Daniel)
- 09:00 - 09:15: Daniel <> Shaked Sync (Attendees: Shaked, Daniel)
- 11:00 - 11:45: Multi-Agent Sync (Attendees: Daniel)
- 12:30 - 14:00: Xpander Assistance Agent (Attendees: Daniel)

**Tasks:**
- High Priority:
  - build an research AI Agent - demo (Due: 2025-02-04)
  - running benchmark to test AI function calls (Due: 2025-02-04)
  - UI bug fix agent builder (Due: 2025-02-04)
  - xpander Assistance agent (Due: 2025-02-04)
- Other Tasks:
  - Update Research Agent (Due: 2025-02-04)

## Latest Updates
1. **Asymmetries of Service: Interdependence and Synchronicity**
   - Source: ArXiv
   - Key Point: Explores dynamics of service interactions between customers and AI agents, offering insights into optimal agent concurrency levels.
   - Link: [Read more](http://arxiv.org/abs/2402.15533v2)

2. **RILe: Reinforced Imitation 